# Homework 6
任课老师：胡俊峰

负责助教：苏亚鲁、李浩然

注意：仅需要提交.ipynb文件，请**不要**将下发压缩包中的其他文件一并交上。

截止日期：3月18日24点

## 第零部分 代码理解
### 0.1 阅读代码预测结果，分析结果是否如预期，并按要求填充代码行，实现功能。

In [1]:
import pandas as pd
df = pd.DataFrame([[1, 2], [3, 4]], columns = ['a','b'])
print(df)
print(df['a']) 

   a  b
0  1  2
1  3  4
0    1
1    3
Name: a, dtype: int64


In [2]:
# 理解那个是0号索引
dft = df.T           
print(dft[1])
print(dft[1]['a'])  #  print(dft['a'][1] ?)
print(dft['a'][1])  #  KeyError, should be [column][row]

a    3
b    4
Name: 1, dtype: int64
3


KeyError: 'a'

In [3]:
df2 = pd.DataFrame([[5, 6], [7, 8]], columns = ['a','b'])
df3 = pd.concat([df,df2])  # 尝试用一行代码，df1，df2的横向拼接？
df4 = pd.concat([df,df2],axis=1)
print(df3)
print(df4)

   a  b
0  1  2
1  3  4
0  5  6
1  7  8
   a  b  a  b
0  1  2  5  6
1  3  4  7  8


In [4]:
# 请填充一行代码，实现打印 ‘b’列的值 <6 的df3的子视图
print(df3[df3['b'] < 6])

   a  b
0  1  2
1  3  4


In [5]:
df = pd.DataFrame([[4, 9]] * 3, columns=['A', 'B'])
print(df)

   A  B
0  4  9
1  4  9
2  4  9


dataframe.apply(function,axis)对一行或一列做出一些操作（axis=1遍历行，axis=0遍历列），可以使用匿名函数。

In [6]:
import numpy as np
print(df.apply(np.sum))
print(df.apply(np.sum, axis=1))

A    12
B    27
dtype: int64
0    13
1    13
2    13
dtype: int64


In [7]:
df.apply(lambda x: [1, 2], axis=1)

0    [1, 2]
1    [1, 2]
2    [1, 2]
dtype: object

In [8]:
df.apply(lambda x: [1, 2], axis=1, result_type='expand')

,0,1
0,1,2
1,1,2
2,1,2


### 0.2 请认真阅读并运行以下示例代码，并思考C/S的多进程、多线程以及下面代码中select实现方式的优缺点。
注：文件夹下的example_client.py为本思考题对应的客户端文件。client.py为第一部分网络编程对应的客户端文件。（两个客户端均无需改动）

In [ ]:
import select
from socket import *
import sys
import queue

# 生成socket对象
server = socket(AF_INET, SOCK_STREAM)
server.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
server.setblocking(False)

# 绑定地址，设置监听
server.bind(('localhost',9999))
server.listen(5)

# 将自己也放进待监测列表里
inputs = [server, ]
outputs = []
message_queues = {}

while True:

    rlist, wlist, elist = select.select(inputs,outputs,inputs) #如果没有任何fd就绪,那程序就会一直阻塞在这里

    for r in rlist:  # 遍历已经可以准备读取数据的 fd
        if r is server: # 如果这个 fd 是server，即 server 有数据待接收读取，说明有新的客户端连接过来了
            conn, client_addr = r.accept()
            print("new connection from",client_addr)
            conn.setblocking(False)
            inputs.append(conn) # 将这个新的客户端连接添加到检测的列表中
            message_queues[conn] = queue.Queue() # 用队列存储客户端发送来的数据，等待服务器统一返回数据

        else:          # 这个可读的 r 不是服务器，那就是某个客户端。就是说客户端发送数据过来了，这些数据处于待读取状态
            try:       # 异常处理，这是为了防止客户端异常断开报错（比如手动关掉客户端黑窗口，服务器也会跟着报错退出）
                data = r.recv(1024)
                if data:    # 根据判断data是否为空，判断客户端是否断开
                    print("收到来自[%s]的数据:" % r.getpeername()[0], data)
                    message_queues[r].put(data)   # 收到的数据先放到queue里,一会返回给客户端
                    if r not in outputs:
                        outputs.append(r)     # 放进可写的fd列表中，表明这些 fd 已经准备好去发送数据了。
                else:   # 如果数据为空，表明客户端断开了
                    print('客户端断开了')
                    if r in outputs:
                        outputs.remove(r)    #  清理已断开的连接
                    inputs.remove(r)         # 清理已断开的连接
                    del message_queues[r]    # 清理已断开的连接
            except ConnectionResetError:     # 如果报错，说明客户端断开了
                print("客户端异常断开了", r)
                if r in outputs:
                    outputs.remove(r)   # 清理已断开的连接
                inputs.remove(r)        # 清理已断开的连接
                del message_queues[r]  # 清理已断开的连接

    for w in wlist:       # 遍历可写的 fd 列表，即准备好发送数据的那些fd
        # 判断队列是否为空
        try :
            next_msg = message_queues[w].get_nowait()
        except queue.Empty:
            # print("client [%s]" % w.getpeername()[0], "queue is empty..")
            outputs.remove(w)
        # 队列不为空，就把队列中的数据改成大写，原样发回去
        else:
            # print("sending msg to [%s]"% w.getpeername()[0], next_msg)
            w.send(next_msg.upper())

    for e in elist:   # 处理报错的 fd
        e.close()
        print("Error occured in ",e.getpeername())
        inputs.remove(e)
        if e in outputs:
            outputs.remove(e)
        del message_queues[e]

new connection from ('127.0.0.1', 53445)
收到来自[127.0.0.1]的数据: b'GET / HTTP/1.1\r\nHost: 127.0.0.1:9999\r\nConnection: keep-alive\r\nsec-ch-ua: "Not_A Brand";v="8", "Chromium";v="120", "Microsoft Edge";v="120"\r\nsec-ch-ua-mobile: ?0\r\nsec-ch-ua-platform: "Windows"\r\nUpgrade-Insecure-Requests: 1\r\nUser-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36 Edg/120.0.0.0\r\nAccept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7\r\nSec-Fetch-Site: none\r\nSec-Fetch-Mode: navigate\r\nSec-Fetch-User: ?1\r\nSec-Fetch-Dest: document\r\nAccept-Encoding: gzip, deflate, br\r\nAccept-Language: en,zh-CN;q=0.9,zh;q=0.8,en-GB;q=0.7,en-US;q=0.6,no;q=0.5,zh-TW;q=0.4\r\nCookie: _ga=GA1.1.1364373718.1688716276; _xsrf=2|a2c79a0a|0900834bd4f3808b23f7263654e177db|1689730844; _ga_R1FN4KJKJH=GS1.1.1694929488.21.1.1694930933.0.0.0\r\n\r\n'
客户端断开了
new connection from (

## 第一部分：网络编程 (2.5分)

### 请将下列代码补充完整，从而实现如下功能：
**功能：**
* 为每个socket注册用户名，并检查用户名是否已经存在。若已经存在，则发送'Username already exists!'；否则发送'Welcome!'。(1.5分)
* 将注册成功的用户发送的信息，转发给其他的注册成功的用户。(1分)

**注：**
当补充完整代码后，可以运行client.py进行检验，其预期的输出为：\
connecting to localhost port 服务器端口 \
Welcome!to 第一个客户端端口 \
Username already exists!to 第二个客户端端口 \
Welcome!to 第二个客户端端口 \
Hello everybody!to 第二个客户端端口 

In [ ]:
from socket import *
from select import select
import queue

class Server:
    def __init__(self, host, port):
        self.server = socket(AF_INET, SOCK_STREAM)
        self.server.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
        self.server.setblocking(False)

        self.server.bind((host, port))
        self.server.listen(5)
        self.rlist = [self.server]
        self.wlist = []
        self.message_queues = {}
        self.users = {}  # 用于判断socket是否已经注册用户名

    def accept(self):
        while True:
            rs, ws, xs = select(self.rlist, self.wlist, self.rlist)
            for r in rs:
                if r is self.server:
                    # 服务器接受客户端连接
                    conn, addr = self.server.accept()
                    print("new connection from",addr)
                    conn.setblocking(False)
                    self.rlist.append(conn)
                    self.message_queues[conn] = queue.Queue()
                    self.users[conn] = False
                else:
                    try:
                        # 服务器接受客户端的消息并转发给所有客户端
                        data = r.recv(1024)
                        if data:
                            print("收到来自[%s]的数据:" % r.getpeername()[1], data.decode('utf-8'))
                            self.wlist.append(r)
                            if self.users[r]:  # 将已注册用户的信息广播给其他已注册用户
                                #print("%sregistered" % r.getpeername()[1])
                                for i in self.users:
                                    if i != r:
                                        self.wlist.append(i)
                                        self.message_queues[i].put(data)
                            else:  # 对未注册用户发送的昵称进行检验，并进行注册
                                #print("%s not registered" % r.getpeername()[1])
                                if data not in self.users.values():
                                    #print("new")
                                    self.users[r] = data
                                    self.message_queues[r].put("Welcome!".encode("utf-8"))
                                else:
                                    #print("used")
                                    self.message_queues[r].put("Username already exists!".encode("utf-8"))
                        else:
                            print('客户端断开了')
                            if r in self.wlist:
                                self.wlist.remove(r)   # 清理已断开的连接
                            self.rlist.remove(r)        # 清理已断开的连接
                            del self.message_queues[r]  # 清理已断开的连接
                    except ConnectionResetError:
                        print("客户端异常断开了", r)
                        if r in self.wlist:
                            self.wlist.remove(r)   # 清理已断开的连接
                        self.rlist.remove(r)        # 清理已断开的连接
                        del self.message_queues[r]  # 清理已断开的连接
            for w in ws:       # 遍历可写的 fd 列表，即准备好发送数据的那些fd
                # 判断队列是否为空
                try :
                    next_msg = self.message_queues[w].get_nowait()
                except queue.Empty:
                    # print("client [%s]" % w.getpeername()[0], "queue is empty..")
                    self.wlist.remove(w)
                # 队列不为空，就把队列中的数据改成大写，原样发回去
                else:
                    print("发送数据到 [%s]"% w.getpeername()[0], next_msg)
                    w.send(next_msg)

            for e in xs:   # 处理报错的 fd
                e.close()
                print("Error occured in ",e.getpeername())
                self.rlist.remove(e)
                if e in self.wlist:
                    self.wlist.remove(e)
                del self.message_queues[e] 

if __name__=='__main__':
    host = '0.0.0.0'      # 主机名
    port = 9999           # 端口号
    server = Server(host,port)
    server.accept()

## 第二部分 Pandas基础练习（4.5分）

### 2.1 基本操作

In [ ]:
import pandas as pd
import numpy as np

data = {
    'age': [24, 53, 23, 24, 33, 42, 57, 57, 36, 29, 53, 39],
    'gender': ['M', 'F', 'M', 'M', 'F', 'M', 'M', 'M', 'M', 'M', 'M', 'F'],
    'occupation': ['technician', 'other', 'writer', 'technician', 'technician', 'executive', 'administrator', 'administrator', 'administrator', 'student', 'lawyer', 'other'],
    'zip_code': ['85711', '94043', '32067', '43537', '15213', '98101', '91344', '91344', '05201', '01002', '90703', '30329']
}

创建一组从1开始递增的id，作为行索引，由data生成一个DataFrame。并以gender分组，打印每组的前两条。（0.5分）

In [ ]:
# TODO
df1 = pd.DataFrame(data,index=range(1,13))
#print(df1)
#print([i for i in range(1,13) if (df1['gender'] == 'M')[i] == True])
print(df1[df1['gender'] == 'M'][:2])
print(df1[df1['gender'] == 'F'][:2])

删除DataFrame中重复的行（保留一行），再删除所有occupation为"other"的行。（0.5分）

In [ ]:
# TODO
df1 = df1.drop([i for i in df1.index if (df1['occupation'] == 'other')[i] == True])
df1 = df1.drop_duplicates(['zip_code'])

#print([i for i in range(1,13) if (df1['occupation'] == 'other')[i] == True])
print(df1)

以两种方式访问DataFrame的第一行。（0.5分）

In [ ]:
# TODO
print(df1.loc[1:1])
print(df1.iloc[0:1])
print(df1[0:1])

print(df1.loc[1])
print(df1.iloc[0])

以两种方式访问DataFrame的"occupation"列。（0.5分）

In [ ]:
# TODO
print(df1.loc[:,"occupation":"occupation"],end='\n\n')
print(df1.iloc[:,2:3],end='\n\n')
print(df1["occupation"])

打印出DataFrame中出现次数最多的职业名称（0.5分）

In [ ]:
# TODO
m = max(res:=df1["occupation"].value_counts())
#print(m)
for i in df1["occupation"].unique():
    if res.loc[i] == m:
        #print(res)
        print(i)


将data_2转换成DataFrame，与上一个DataFrame按照"zip_code"这一列合并，按两种合并方式：（0.5分）

1.合并后结果保留两个DataFrame的并集；

2.合并后只保留data_2的所有数据。

In [ ]:
data_2 = {
    'zip_code': ['85711', '94043', '15213', '98101', '30329'],
    'name': ['Alan', 'Alice', 'Abigail', 'Bob', 'Dave']
}
df2 = pd.DataFrame(data_2)

# TODO
df3 = pd.merge(df1, df2, how="outer", on="zip_code")
df4 = pd.merge(df1, df2, how="right", on="zip_code")
print(df3)
print(df4) # 之前df1里删除的这里应该没有吧 94043和30329的数据已经在前面删掉了 不会扣分吧:O

### 2.2 多级索引

In [ ]:
# 构建多级索引DataFrame
# int数据由 np.random.randint(0, 150, size=(6, 3) 产生
# from tuples, from_arrays, from_product
import numpy as np
import pandas as pd

df3 = pd.DataFrame(np.random.randint(0, 150, size=(6, 3)),
                  columns=["Python", "C++", "Java"],
                  index=pd.MultiIndex.from_product([['张三', '李四', '王五'],
                                                  ['期中', '期末']]))
df3


实现变换df3为三重索引的serial。(0.5分)

In [ ]:
# TODO
df4 = df3.stack()
df4
#df4.index.names = ['name','period','course']
#df4.swaplevel('course','period')


按期中40%，期末60%的比例，计算输出每位同学每门功课的总评成绩表。（0.5分）

In [ ]:
# TODO
#df5 = pd.DataFrame({'Python':[0,0,0],'C++':[0,0,0],'Java':[0,0,0]}, index=['张三','李四','王五'])
data = {}
def calc(a,b):
    return a*0.4 + b*0.6
for i in df3.columns:
    for j in df3.index:
        data[j[0],i] = calc(*tuple(df3[i][j[0]]))
print(df5:=pd.Series(data).unstack())

计算输出每门功课的最高分，最低分，以及平均分数表（课程名做行索引，列名用最高分，最低分，平均分数）。（0.5分）

In [ ]:
# TODO
data = {}
for i in df5.columns:
    data[i] = [max(df5[i]), min(df5[i]), round(np.average(df5[i]),1)]
print(pd.DataFrame(data, index=["最高分","最低分","平均分数"]).T)

## 第三部分 Pandas数据分析（3分）

使用Pandas的read_csv函数读取 IBM_2006-01-01_to_2018-01-01.csv 中的数据，得到一个DataFrame，通过shape和info()查看数据基本信息，并打印该数据的前10行观察一下。该数据列索引对应的分别是：日期，开盘价，最高价，最低价，收盘价，成交量，股票名称。（0.5分）

In [ ]:
import pandas as pd

IBM_file = 'IBM_2006-01-01_to_2018-01-01.csv'
# TODO
df1 = pd.read_csv(IBM_file)
print(df1.shape)
print(df1.info)
print(df1[:][0:10])

观察到Open与Low这两列相比其他列都缺了一项，请删除DataFrame中所有不完整的数据。（0.5分）

In [ ]:
# TODO
df1.drop(df1[df1["Open"].isna()].index) #No.2913 found

由于股市周末和节假日不开盘，日期为2006-01-06的下一条数据就是2006-01-09，缺少了中间两天的数据。请将所有缺少的数据按照以下规则补充：成交量（volume）填充为零，开盘价等价格与前一天（最近的有数据的一天）保持相同。（0.5分）

提示：使用pandas.date_range生成时间序列

In [ ]:
# TODO
df1Dates = list(df1['Date'])
dateRange = pd.date_range(start='2006-01-03', end='2018-01-01')
x = -1
for i in dateRange:
    x += 1
    if (d:=str(i)[:10]) not in df1Dates:
        df1 = pd.DataFrame(np.insert(df1.values, x, values=[d] + list(df1.iloc[x-1,1:5]) + [0] + [df1.iloc[x-1,6]], axis=0),columns=['Date','Open','High','Low','Close','Volume','Name'])
df1

在DataFrame中按月份聚合，计算一年当中每个月的成交量总和，以及每个月收盘价的次高值。(1分)

提示：可以通过apply方法实现自定义的聚合函数。（有多个相同值的话，这里先简单地按照排完序取下标第二个就行了）

In [ ]:
# TODO
sepa_head = [pd.date_range(start='2006-01-03', end='2006-01-03')] + list(pd.date_range(start='2006-01-03', end='2017-12-31',freq='MS'))
sepa_tail = list(pd.date_range(start='2006-01-03', end='2017-12-31',freq='M'))
dates = list(pd.date_range(start='2006-01-03', end='2017-12-31'))
data = {}
for i in range(len(sepa_head)):
    #print(dates.index(sepa_head[i]),dates.index(sepa_tail[i]))
    dfMonth = df1.loc[dates.index(sepa_head[i]):dates.index(sepa_tail[i]),'Volume']
    #print(sum(dfMonth))
    #print(list(dfMonth.sort_values())[-2])
    data[str(sepa_head[i])[:7]] = [sum(dfMonth), list(dfMonth.sort_values())[-2]]
print(pd.DataFrame(data, index=["成交量总和","收盘价次高值"]).T)

读取AMZN_2006-01-01_to_2018-01-01.csv，其中包含了Amazon的股票数据。
请以Date作为索引，'IBM'和'AMZN'作为列索引，构建一个Dataframe，其中的数据为两家公司每天的股票收盘价。（0.5分）

In [ ]:
AMZN_file = 'AMZN_2006-01-01_to_2018-01-01.csv'
IBM_file = 'IBM_2006-01-01_to_2018-01-01.csv'

AMZN_data = pd.read_csv(AMZN_file)
IBM_data = pd.read_csv(IBM_file)

# TODO
AMZN_part = pd.DataFrame(AMZN_data, columns=['Date', 'Close'])
IBM_part = pd.DataFrame(IBM_data, columns=['Date', 'Close'])
df2 = pd.merge(IBM_part, AMZN_part, how='outer', on='Date')
df2 = df2.set_index('Date')
df2 = df2.set_axis(['IBM',"AMZN"], axis=1)
print(df2)